In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


True
1
NVIDIA GeForce RTX 5080


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "deepseek-ai/deepseek-llm-7b-chat"  # You can use a smaller DeepSeek if limited on RAM/VRAM

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True
)
#    offload_folder="/tmp/offload"  # or any writable folder
#)



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
prompt = "Hello, how are you today?"


In [2]:
print(next(model.parameters()).device)
import torch
print(torch.cuda.is_available())


cuda:0
True


In [7]:
# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,        # For more natural conversation
    top_p=0.9,             # Sampling parameters
    temperature=0.7
)

# Decode and print
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Hello, how are you today?
Hello, I am good, and you?
I am also good, thank you.
The weather is beautiful today, isn't it?
Yes, the weather is beautiful today. The sun is shining and the sky is clear.
It's a perfect day for a walk or a picnic.
Yes, it definitely is. I'm planning to go for a hike later.
That sounds great. Where are you planning to go hiking?
I'm planning to go to the mountains. The view from up there is breathtaking.
That sounds wonderful. Have a great time.
Thank you,


In [ ]:
import gradio as gr

def chat(message, history):
    print('history ' + str(history) + ' message ' + str(message))
    prompt = ""
    for turn in history:
        if len(turn) == 2:
            prompt += f"User: {turn[0]}\nAssistant: {turn[1]}\n"
    prompt += f"User: {message}\nAssistant:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.98,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response[len(prompt):].strip()
    return answer

gr.ChatInterface(chat).launch(share=False)



/home/jijunjiang/DeepSeek-LLM/venv/lib/python3.12/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


history [] message tell me a long story
history [['tell me a long story', "Once upon a time, there was a young man named Jack who lived in a small village on the edge of a dense forest. Jack was a kind and hard-working man who was known throughout the village for his incredible strength and his ability to fix almost anything.\n\nOne day, while out in the forest gathering wood, Jack stumbled upon an old, rusted iron chest. Intrigued, he picked it up and carried it back to his house. Over the next few days, Jack couldn't stop thinking about the chest and what might be inside it. Finally, he decided to open it.\n\nTo his surprise, the chest was filled with gold coins and precious gems. Jack knew that he couldn't keep the treasure for himself, so he decided to use it to help those in need. He used the money to build a school for the children of the village and a hospital for the sick.\n\nWord of Jack's generosity spread throughout the land, and soon people from all over were coming to the 

: 